# Image Generator

In [ ]:
import cv2
import numpy as np
import random
import os
from numba import njit


@njit
def random_point(x, y, margin=30):
    return [x + random.randint(-margin, margin), y + random.randint(-margin, margin)]


def rotate_image(image, angle, background_color):
    height, width, _ = image.shape
    center = (width // 2, height // 2)
    rotation_matrix = cv2.getRotationMatrix2D(center, angle, 1.0)
    return cv2.warpAffine(
        image,
        rotation_matrix,
        (width, height),
        borderMode=cv2.BORDER_CONSTANT,
        borderValue=background_color,
    )


def skew_image(image, background_color):
    height, width, _ = image.shape
    src_pts = np.float32([[0, 0], [width, 0], [0, height], [width, height]])
    dst_pts = np.float32([
        random_point(0, 0),
        random_point(width, 0),
        random_point(0, height),
        random_point(width, height),
    ])
    matrix = cv2.getPerspectiveTransform(src_pts, dst_pts)
    return cv2.warpPerspective(
        image,
        matrix,
        (width, height),
        borderMode=cv2.BORDER_CONSTANT,
        borderValue=background_color,
    )


def precomputed_sin_lut(size, strength):
    return (strength * np.sin(np.linspace(0, 2 * np.pi, size, endpoint=False))).astype(
        np.float32
    )


def warp_image(image, background_color, warp_strength=25, axis="x", lut=None):
    height, width, _ = image.shape
    x, y = np.meshgrid(
        np.arange(width, dtype=np.float32), np.arange(height, dtype=np.float32)
    )
    if lut is None:
        lut = precomputed_sin_lut(height if axis == "x" else width, warp_strength)
    if axis == "x":
        offset = lut[(y[:, 0].astype(int)) % len(lut)].reshape(-1, 1)
        x += offset
    else:
        offset = lut[(x[0, :].astype(int)) % len(lut)].reshape(1, -1)
        y += offset
    np.clip(x, 0, width - 1, out=x)
    np.clip(y, 0, height - 1, out=y)
    return cv2.remap(
        image,
        x,
        y,
        interpolation=cv2.INTER_LINEAR,
        borderMode=cv2.BORDER_CONSTANT,
        borderValue=background_color,
    )


def random_color():
    return [random.randint(0, 255) for _ in range(3)]


def generate_contrasting_color(shape_color):
    while True:
        bg_color = random_color()
        if all(abs(shape_color[i] - bg_color[i]) >= 10 for i in range(3)):
            return bg_color


def generate_shape_image(shape, size=(400, 400)):
    bg_color = random_color()
    canvas = np.full((size[0], size[1], 3), bg_color, dtype=np.uint8)
    shape_color = generate_contrasting_color(bg_color)

    max_x = size[1] - 1
    max_y = size[0] - 1

    scale = random.uniform(0.45, 1.05)
    angle = random.randint(0, 360)

    buffer = 40
    max_x -= buffer
    max_y -= buffer

    if shape == "circle":
        radius = int((min(size) // 4) * scale)
        offset_x = random.randint(radius + buffer, max_x - radius)
        offset_y = random.randint(radius + buffer, max_y - radius)

        circle_center = (offset_x, offset_y)
        cv2.circle(canvas, circle_center, radius, shape_color, 4)
    elif shape == "square":
        side = int((min(size) // 2) * scale)
        diagonal = int(np.sqrt(2) * side)

        offset_x = random.randint(diagonal // 2 + buffer, max_x - diagonal // 2)
        offset_y = random.randint(diagonal // 2 + buffer, max_y - diagonal // 2)

        top_left = (offset_x - side // 2, offset_y - side // 2)
        bottom_right = (offset_x + side // 2, offset_y + side // 2)
        cv2.rectangle(canvas, top_left, bottom_right, shape_color, 2)
    elif shape == "triangle":
        side = int((min(size) // 2) * scale)
        height = int(np.sqrt(3) / 2 * side)
        base = side

        offset_x = random.randint(base // 2 + buffer, max_x - base // 2)
        offset_y = random.randint(height // 2 + buffer, max_y - height // 2)

        points = np.array(
            [
                [offset_x, offset_y - height // 2],
                [offset_x - base // 2, offset_y + height // 2],
                [offset_x + base // 2, offset_y + height // 2],
            ],
            np.int32,
        )
        cv2.polylines(canvas, [points], isClosed=True, color=shape_color, thickness=4)

    rotated_canvas = rotate_image(canvas, angle, bg_color)
    skewed_canvas = skew_image(rotated_canvas, bg_color)
    warped_canvas = warp_image(
        skewed_canvas, bg_color, warp_strength=10, axis=random.choice(["x", "y"])
    )
    return warped_canvas


def save_images(num_images=10, output_folder="shapes"):
    os.makedirs(output_folder, exist_ok=True)
    shapes = ["circle", "square", "triangle"]
    shape_counts = {"circle": 0, "square": 0, "triangle": 0}

    for shape in shapes:
        os.makedirs(os.path.join(output_folder, shape), exist_ok=True)

    for _ in range(num_images):
        shape = random.choice(shapes)
        shape_counts[shape] += 1
        img = generate_shape_image(shape)
        filename = os.path.join(output_folder, shape, f"{shape_counts[shape]}.png")
        cv2.imwrite(filename, img)
        print(f"Saved {filename}")


if __name__ == "__main__":
    save_images(30000, "training")
    save_images(1000, "testing")


# Model Trainer

In [ ]:
import cv2
import numpy as np
import os
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split


def load_data(image_folder, image_size=(128, 128)):
    progress = 0
    progress_hun = 0
    images = []
    labels = []
    shape_labels = {"circle": 0, "square": 1, "triangle": 2}

    for shape in shape_labels.keys():
        shape_folder = os.path.join(image_folder, shape)
        for filename in os.listdir(shape_folder):
            progress += 1
            if filename.endswith(".png"):
                img = cv2.imread(
                    os.path.join(shape_folder, filename), cv2.IMREAD_COLOR_RGB
                )
                img = cv2.resize(img, image_size)
                images.append(img)
                labels.append(shape_labels[shape])
            if progress - (progress % 100) > progress_hun:
                progress_hun = progress
                print(f"Progress: {progress_hun} images processed.")

    images = np.array(images)
    labels = np.array(labels)

    images = images.astype("float32") / 255.0

    images = np.expand_dims(images, axis=-1)

    labels = to_categorical(labels, num_classes=3)

    return images, labels


train_images, train_labels = load_data("training")
test_images, test_labels = load_data("testing")

x_train, x_val, y_train, y_val = train_test_split(
    train_images, train_labels, test_size=0.2, random_state=42
)

model = Sequential([
    Conv2D(32, (3, 3), activation="relu", input_shape=(128, 128, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation="relu"),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation="relu"),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(256, activation="relu"),
    Dense(3, activation="softmax"),
])

model.compile(
    optimizer=Adam(learning_rate=0.0009),
    loss="categorical_crossentropy",
    metrics=["accuracy"],
)

early_stop = EarlyStopping(
    monitor="val_accuracy", patience=2, restore_best_weights=True
)

checkpoint = ModelCheckpoint(
    "shape_recognition_model.keras",
    monitor="val_accuracy",
    save_best_only=True,
    mode="max",
    verbose=1,
)

model.fit(
    x_train,
    y_train,
    validation_data=(x_val, y_val),
    epochs=12,
    batch_size=32,
    callbacks=[early_stop, checkpoint],
)

test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f"Test accuracy: {test_acc}")

model.save("./dist/shape-sorter.keras")


# Loader

In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model


def preprocess_image(image_path, image_size=(128, 128)):
    img = cv2.imread(image_path, cv2.IMREAD_COLOR_RGB)

    if img is None:
        print(f"Error: The image at {image_path} could not be loaded.")
        return None

    img = cv2.resize(img, image_size)
    img = img.astype("float32") / 255.0
    img = np.expand_dims(img, axis=-1)
    img = np.expand_dims(img, axis=0)

    return img


def predict_shape(image_path, model_path="./dist/shape-sorter.keras"):
    model = load_model(model_path)
    img = preprocess_image(image_path)

    if img is None:
        return

    predictions = np.array(model.predict(img))

    print(f"Circle: {predictions[0][0] * 100}%")
    print(f"Square: {predictions[0][1] * 100}%")
    print(f"Triangle: {predictions[0][2] * 100}%")


if __name__ == "__main__":
    predict_shape("./testing/test.png")
